## Sample execution of the repository

The config file must be set accordingly. The default directs to the data folder, where the minimal example will be downloaded to and other files generated as well. This can be easily set to an external location if the data become large.

Then, a minimal working example file can be downloaded from ftp://magftp.gfz-potsdam.de/PyPlatMag/ for December (12) of 2009 for the GOCE satellite mission to showcase the use of the repository.

In [ ]:
import urllib.request
import os
from pathlib import Path

def download_file(url, dir, filename):
    if not os.path.exists(dir):
        os.makedirs(dir)
    print(f"Downloading {filename}...")
    filename = dir + filename
    urllib.request.urlretrieve(url, filename)
    print(f"Downloading {filename} completed")

dirname = os.path.abspath('')
print("prev dirname: ", dirname)
orbit_path = os.path.join(dirname, 'data/orbit/')
goce_path = os.path.join(dirname, 'data/raw/')
write_path = os.path.join(dirname, 'data/intermediate/')
download_file('ftp://magftp.gfz-potsdam.de/PyPlatMag/GO_data_200912.h5', goce_path, 'GO_data_200912.h5')
download_file('ftp://magftp.gfz-potsdam.de/PyPlatMag/GO_rawdata_200912.h5', goce_path, 'GO_rawdata_200912.h5')
download_file('ftp://magftp.gfz-potsdam.de/PyPlatMag/GO_rawdata_hk_200912.h5', goce_path, 'GO_rawdata_hk_200912.h5')
download_file('ftp://magftp.gfz-potsdam.de/PyPlatMag/GO_rawdata_telemetry_200912.h5', goce_path, 'GO_rawdata_telemetry_200912.h5')
download_file('ftp://magftp.gfz-potsdam.de/PyPlatMag/GO_ORBCNT_20090317T231001_20131109T234516_0101.h5', orbit_path, 'GO_ORBCNT_20090317T231001_20131109T234516_0101.h5')


Next, the required CDF-binaries are installed. You can skip this step if you have your cdf-installation already at a different place.

In [ ]:
# Optional: CDF(-binary) installation
!wget https://spdf.gsfc.nasa.gov/pub/software/cdf/dist/latest/linux/cdf39_0-dist-all.tar.gz && \
!tar -xvf cdf39_0-dist-all.tar.gz
!cd ./cdf39_0-dist
!make OS=linux ENV=gnu CURSES=yes all
!make test
!make install
!make clean

In [ ]:
import ruamel.yaml
yaml = ruamel.yaml.YAML()
with open('config.yaml') as fp:
    data = yaml.load(fp)
    data['goce_data_path'] = goce_path
    data['write_path'] = write_path
    data['orbit_counter_path'] = orbit_path + 'GO_ORBCNT_20090317T231001_20131109T234516_0101.h5'
    ret = ruamel.yaml.comments.CommentedSeq()
    ret.fa.set_flow_style()
    ret.append('200912')
    data['year_month_specifiers'] = ret
    cdf_location_bin = os.path.join(dirname, "cdf39_0-dist/bin/")
    cdf_location_lib = os.path.join(dirname, "cdf39_0-dist/lib/")
    data['CDF_BIN'] = cdf_location_bin
    data['CDF_LIB'] = cdf_location_lib

with open('config.yaml', 'wb') as fp:
    yaml.dump(data, fp)

with open('config_goce.yaml') as fp:
    data = yaml.load(fp)
    data['cdf_config']['cdf_path'] = 'data/cdf/'
    data['train_config']['learn_config']['epochs'] = 20
    data['train_config']['learn_config']['epochs_finetune'] = 20


with open('config_goce.yaml', 'wb') as fp:
    yaml.dump(data, fp)

After setting the config file accordingly, the processing can begin with the data reading, combining, and enriching: read_files()

In [ ]:
from scripts.read_files import read_files

read_files()

Mean filling for NaNs is realized in read_files_nan_handling()

In [ ]:
from scripts.read_files_nan_handling import read_files_nan_handling

read_files_nan_handling()

With this (final) intermediate dataset in place, the calculation of the global ML model for the calibration can begin:

In [ ]:
from scripts.train_pinn import train_pinn

train_pinn()

Finetune the global model for single months:

In [ ]:
from scripts.train_pinn_finetune import train_pinn_finetune

train_pinn_finetune()

Finally, CDF files containing the calibration results are generated utilizing the finetunde monthly models.

In [ ]:
from scripts.generate_cdf_files import generate_cdf_files

generate_cdf_files()